In [ ]:
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so

from sklearn import linear_model    # Herramientas de modelos lineales
from sklearn.metrics import mean_squared_error, r2_score    # Medidas de desempeño
from sklearn.preprocessing import PolynomialFeatures    # Herramientas de polinomios

from formulaic import Formula

In [ ]:
# Si necesitan instalar algún paquete
#pip install gapminder
#pip install formulaic

## Introducción: sistemas de ecuaciones lineales

Sabemos que un fonde de inversión invirtió en acciones de YPF, Santander y Nvidia (y solo en estas acciones) pero no sabemos cuántas acciones compró de cada una. ¿Cómo podemos averiguarlo? 

Suponemos que tenemos disponible:
1. La valorización del fondo al final de cada día.
2. El valor la acción de cada empresa al cierre de cada día.

In [ ]:
# Cargamos los datos
dataDict = {'total': [170262,169929.5,171064,169637.35,164625.45], 
        'YPF': [20935, 21030, 20770, 20950, 20750], 
        'Santander': [20100, 20500, 21700, 21000, 20316], 
        'Nvidia': [37100, 36255, 36000, 35645.5, 33878.5]}
data = pd.DataFrame.from_dict(dataDict)
data

Nos quedamos con las primeras tres filas y resolvemos el sistema lineal:
$$total = c_1 \cdot YPF + c_2 \cdot Santander + c_3 \cdot Nvidia$$


In [ ]:
data_3rows = data[[True, True, True, False, False]]
data_3rows

In [ ]:
X_3rows = data_3rows[["YPF", "Santander", "Nvidia"]]
y_3rows = data_3rows["total"]
display(X_3rows)
display(y_3rows)

Para obtener los valores de c_1, c_2 y c_3 resolvemos el sistema lineal utilizando `np.linalg.solve`

In [ ]:
c = np.linalg.solve(X_3rows, y_3rows)
print(c)

In [ ]:
# Verificamos
X_3rows @ c

**Pregunta:** A partir de la respuesta, ¿podemos confirmar que este fondo tenía estas 3 acciones? Si no estamos seguros si eran acciones de Santander o Galicia, ¿cómo podríamos asegurarnos?

¿Qué pasa si hacemos lo mismo usando los valores de Galicia en lugar de Santander?

In [ ]:
# Incorporamos una nueva columna al dataframe
data["Galicia"] = np.array([20100, 19400, 21900, 20600, 20900])
data

Resolvemos el sistema de 3 x 3 utilizando los datos de Galicia

In [ ]:
dataG_3rows = data[[True, True, True, False, False]]
XG_3rows = dataG_3rows[["YPF", "Galicia", "Nvidia"]]
yG_3rows = dataG_3rows["total"]
np.linalg.solve(XG_3rows, yG_3rows)

Un sistema de 3x3 en general va a tener solución (única), no nos sirve para ver si nuestro "modelo" es correcto.
Podemos hacer dos cosas:
    
1. Verificar la formula obtenida en los dias siguientes.
2. Utilizar todos los días disponibles para encontrar los coeficientes.

Para el método 1 separamos nuestro DataFrame en dos: conjunto de entrenamiento (las primeras 3 filas) y conjunto de testeo (las últimas 2 filas).

In [ ]:
train_ind = np.array([True, True, True, False, False])
test_ind = np.array([False, False, False, True, True])  # O podemos usar np.logical_not(train_ind)
data_train = data[train_ind]
data_test = data[test_ind]

In [ ]:
# Entrenamos el primer modelo  (usando la variable Santander)
X_train = data_train[["YPF", "Santander", "Nvidia"]]
y_train = data_train["total"]
c = np.linalg.solve(X_train, y_train)
c

In [ ]:
# Verificamos si nuestro modelo "predice" bien en el conjunto de testeo:
X_test = data_test[["YPF", "Santander", "Nvidia"]]
y_test = data_test["total"]
X_test @ c - y_test # Errores

In [ ]:
# Repetimos con el modelo Galicia.

# Entrenamos el modelo
XG_train = data_train[["YPF", "Galicia", "Nvidia"]]
yG_train = data_train["total"]
cG = np.linalg.solve(XG_train, yG_train)
cG

In [ ]:
# Verificamos si nuestro modelo "predice" bien en el conjunto de testeo:
XG_test = data_test[["YPF", "Galicia", "Nvidia"]]
yG_test = data_test["total"]
XG_test @ cG - yG_test # Errores

Comparando los errores en ambos modelos, vemos que en el primer modelo los errores en las predicciones son nulos y en el segundo no, por lo tanto elegimos el modelo 1.

**Pregunta.** En este ejemplo, cual de los dos modelos tiene errores menores en el ajuste de los datos de entrenamiento?

Ahora consideramos 10 días, y utilizamos los 8 primeros para entrenar el modelo.

In [ ]:
dataDict = {'total': [166650.00, 167756.00, 167981.00, 168597.00, 168796.00, 170262,169929.5,171064,169637.35,164625.45], 
        'YPF': [20100, 20230, 20300, 20410, 20500, 20935, 21030, 20770, 20950, 20750], 
        'Santander': [19800, 19890, 19950, 20150, 20080, 20100, 20500, 21700, 21000, 20316], 
        'Nvidia': [36900, 37200, 37130, 37050, 37080, 37100, 36255, 36000, 35645.5, 33878.5], 
        'Galicia': [23000, 22900, 21800, 21100, 20750, 20100, 19400, 21900, 20600, 20900]}
data = pd.DataFrame.from_dict(dataDict)
data

In [ ]:
train_ind = np.array([True, True, True, True, True, True, True, True, False, False])
test_ind = np.logical_not(train_ind)
data_train = data[train_ind]
data_test = data[test_ind]

In [ ]:
# 1. Queremos calcular los coeficientes utilizando los 8 días
X = data[["YPF", "Santander", "Nvidia"]]
y = data["total"]
scipy.linalg.solve(X, y)

In [ ]:
# Solve nos da error, solo acepta matrices cuadradas, porque un sistema con mas filas en general no tiene solución.
# Cuando tenemos mas filas que columnas usamos lstsq (minimos cuadrados). 
# lstsp minimiza el error cuadradito. Si el sistema tiene solución exacta nos va a dar esa solución.
X_train = data_train[["YPF", "Santander", "Nvidia"]]
y_train = data_train["total"]
c = scipy.linalg.lstsq(X_train, y_train)
c[0] # Los coeficientes 

In [ ]:
# Verificamos en los datos de testeo
X_test = data_test[["YPF", "Santander", "Nvidia"]]
y_test = data_test["total"]
X_test @ c[0] - y_test # Errores

Repetimos el mismo procedimiento con las acciones de Galicia.

In [ ]:
# Solve nos da error, solo acepta matrices cuadradas, porque un sistema con mas filas en general no tiene solución.
# Cuando tenemos mas filas que columnas usamos lstsq (minimos cuadrados). 
# lstsp minimiza el error cuadradito. Si el sistema tiene solución exacta nos va a dar esa solución.
XG_train = data_train[["YPF", "Galicia", "Nvidia"]]
yG_train = data_train["total"]
cG = scipy.linalg.lstsq(XG_train, yG_train)
print("Coeficientes: ", cG[0]) # Los coeficientes 

In [ ]:
# Verificamos en los datos de testeo
XG_test = data_test[["YPF", "Galicia", "Nvidia"]]
yG_test = data_test["total"]
XG_test @ cG[0] - yG_test # Errores

Observamos que los errores son mucho menores en el primer modelo, por lo tanto nos quedamos con ese modelo.

# Caso de estudio: calorías de alimentos

In [ ]:
df_nutricion = pd.read_csv('../data/nutrition.csv')
df_nutricion

Vemos que el DataFrame contiene muchos datos "NaN" (not a number). 

En este ejemplo consideramos que los datos faltantes representan que el alimento no contiene ese ingrediente y lo convertimos a 0.

In [ ]:
# Utilizamos fillna para convertir NaN a 0.
df_nutricion = df_nutricion.fillna(0)

In [ ]:
df_nutricion.head()

Construimos las matrices X e y utilizando Formulaic

In [ ]:
y, X = (
    Formula('Calorias_kcal ~ Proteinas_g + Carbohidratos_g + GrasaTotal_g + Colesterol_mg + Fibra_g + Agua_g + Alcohol_g + VitaminaC_mg')
    .get_model_matrix(df_nutricion)
)

In [ ]:
X.head() # Vemos que nos agregó una columna de Intercept

In [ ]:
y.head()

**Observación**

En este caso sencillo, podemos obtener lo mismo (excepto por la columna Intercept) utilizando una lista de columnas.


In [ ]:
#X = df_nutrition[["Proteinas_g", "Carbohidratos_g", "GrasaTotal_g", "Colesterol_mg","Fibra_g", "Agua_g","Alcohol_g","VitaminaC_mg"]]
#y = df_nutrition["Calorias_kcal"]

Antes de separar en entrenamiento y testeo, veamos los errores del modelo lineal con todos los datos 

In [ ]:
modelo = linear_model.LinearRegression(fit_intercept = False)    # Inicializamos un modelo de Regresion Lineal. 
                                       # Como la matriz X ya tiene el intercept, no agregamos intercept en la regresión
    
modelo.fit(???)   # Realizamos el ajuste

In [ ]:
modelo.coef_

Analizando los coeficientes vemos que las variables Proteinas_g, Carbohidratos_g y GrasaTotal_g son las que tienen mayor peso en el modelo.

Analizamos la "bondad" del ajuste.

In [ ]:
y_pred = modelo.predict(???)
# Calculando el R^2
r2 = r2_score(y, y_pred)
print('R^2: ', r2)

# Calculando el ECM
ecm = mean_squared_error(y, y_pred)
print('Raiz cuadarada del ECM: ', np.sqrt(ecm))

A priori es un buen modelo, tenemos 7792 observaciones y obtenemos R^2 casi igual a 1 con solo 9 variables.

# Conjuntos de entrenamiento y testeo
Ajustamos el modelo separando en 80-20

In [ ]:
from sklearn.model_selection import  

In [ ]:
y, X = (
    Formula('Calorias_kcal ~ Proteinas_g + Carbohidratos_g + GrasaTotal_g + Colesterol_mg + Fibra_g + Agua_g + Alcohol_g + VitaminaC_mg')
    .get_model_matrix(df_comida)
)

# Separamos en entrenamiento (train) y testeo (test).
# El parámetro test_size=0.2 indica que tomamos un 20% de los datos para testeo.
# El parámetro random_state=42 es lo que se denomina semilla aleatoria. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

### Números pseudo-aleatorios y semillas aleatorias.

Las computadoras no pueden generar números al azar, tienen algoritmos que generan números que parecen al azar denominados pseudo aleatorios. 

Los números se generan a partir de una semilla. Si corremos el codigo utilizando la misma semilla,  vamos a obtener siempre el mismo resultado.

Esto permite que el experimento sea reproducible.

### Entrenamiento

In [ ]:
# Entrenamos el modelo utilizando los conjuntos de entrenamiento

modelo = linear_model.LinearRegression(fit_intercept = False)    # Inicializamos un modelo de Regresion Lineal
modelo.fit(???)   # Realizamos el ajuste

### Testeo

In [ ]:
# Medimos la bondad del ajuste en el conjunto de testeo

y_pred = modelo.predict(???)

# Calculando el R^2
r2 = r2_score(???)
print('R^2: ', r2)

# Calculando el ECM
ecm = mean_squared_error(???)
print('Raiz cuadarada del ECM: ', np.sqrt(ecm))

Vemos que el modelo ajusto bien en los datos de testeo, podemos confiar en el modelo obtenido.

**Ejercicio:** Utilizar distintas semillas aleatorias. ¿Se obtienen los mismos valores? ¿Se mantienen las conclusiones?

## Selección de modelos

Comparamos con un modelo utilizando solo las tres primeras variables (las de mayor peso en el modelo) y sin intercept.

In [ ]:
y, X = (
    Formula('Calorias_kcal ~ Proteinas_g + Carbohidratos_g + GrasaTotal_g - 1')
    .get_model_matrix(df_comida)
)
X.head()

In [ ]:
# Separamos en entrenamiento (train) y testeo (test).
# El parámetro test_size=0.2 indica que tomamos un 20% de los datos para testeo.
# El parámetro random_state=42 es lo que se denomina semilla aleatoria. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
modelo = linear_model.LinearRegression(fit_intercept = False)    # Inicializamos un modelo de Regresion Lineal sin intercept
modelo.fit(X_train, y_train)   # Realiza
print("Coeficientes:", modelo.coef_)

y_pred = modelo.predict(X_test)
# Calculando el R^2
r2 = r2_score(y_test, y_pred)
print('R^2: ', r2)

# Calculando el ECM
ecm = mean_squared_error(y_test, y_pred)
print('Raiz cuadarada del ECM: ', np.sqrt(ecm))

Vemos que el modelo es un poco peor pero mucho más simple. 

**Conclusión rápida:** El modelo con 3 variables es útil para una cuenta rápida, pero si necesitamos una mayor precisión podemos utilizar el modelo completo.

**Ejercicio.** Buscar en recursos en-línea la fórmula usualmente utilizada para el cálculo de calorías y comparar con la fórmula que obtuvimos.

# Caso de estudio: rendimiento del suelo

In [ ]:
df_rendimiento = pd.read_csv('../data/rendimiento.csv')
df_rendimiento.head()

**Ejercicio.**

Proponer un modelo para predecir el rendimiento de una hectárea cultivada en función de algunas características del lugar y los fertilizantes utilizados.

In [ ]:
df_rendimiento.info()

In [ ]:
# Acá están todas las variables
"Rendimiento ~ pH + MO + P + N + S + P_Ferti + P_kgha + S_Fuente + S_kgha + Ndisp + Palc + Variedad + FdS + plantas + momento + IncSep + IncMam + IncRoya + FdD + Espigas"

# Interacciones entre variables y la paradoja de Simpson.

Queremos estudiar la relación entre la longitud y la profundidad del pico de los pingüinos.

In [ ]:
penguins = sns.load_dataset("penguins")  # Eliminamos las filas con datos faltantes
penguins.head()

In [ ]:
# Vemos que hay datos faltanes.
# Eliminamos las filas con datos faltantes
penguins = penguins.dropna()  
penguins.head()

In [ ]:
# Ajustamos un modelo lineal y calculamos el coeficiente de correlación R^2
y, X = (
    Formula('???')
    .get_model_matrix(penguins)
)
display(X.head()) # Observamos que eliminó automáticamente las filas con datos faltantes

In [ ]:
modelo = linear_model.LinearRegression(fit_intercept = False)    # Inicializamos un modelo de Regresion Lineal sin intercept
modelo.fit(X, y)   # Realiza
print("Coeficientes:", modelo.coef_)

y_pred = modelo.predict(X)
# Calculando el R^2
r2 = r2_score(y, y_pred)
print('R^2: ', r2)

# Calculando el ECM
ecm = mean_squared_error(y, y_pred)
print('Raiz cuadarada del ECM: ', np.sqrt(ecm))

Si analizamos rápidamente estos resultados diríamos que no hay relación entre el largo y la produndidad... (o que hay correlación negativa porque la pendiente es negativa). Resulta un poco extraño...

¿Cómo podemos analizar mejor qué está pasando?
.

.

.

.

.

.

.

.

.

.

.

.

.

.

.




Realicemos un gráfico!

In [ ]:
(
    so.Plot(data = penguins, x = "bill_length_mm", y = "bill_depth_mm")
    .add(so.Dot())
    .add(so.Line(), so.PolyFit(1))
)

El gráfico confirma la correlación negativa, pero notan algo raro? Tal vez hay algo que no estamos teniendo en cuenta?

.

.

.

.

.


.

.

.

.

.


.

.

.

.

.



Repetimos el gráfico coloreando los puntos según la especie.

In [ ]:
(
    so.Plot(data = penguins, x = "bill_depth_mm", y = "bill_length_mm", color = "species")
    .add(so.Dot())
)

En este gráfico por especie vemos dentro de cada especie puede haber correlación. Verificamos agregando los ajustes por especie.

In [ ]:
(
    so.Plot(data = penguins, x = "bill_depth_mm", y = "bill_length_mm", color = "species")
    .add(so.Dot())
    .add(so.Line(), so.PolyFit(1))
)

Ahora las rectas tienen pendiente positiva! Al considerar todas las especies al mismo tiempo, no podíamos ver esta correlación.

## La paradoja de Simpson
La paradoja de Simpson es un fenómeno estadístico en el cual una relación entre variables aparece, desaparece o se revierte al dividir a la población en subpoblaciones.

**Ejemplo.** Veamos otro ejemplo simulado.
Generamos dos poblaciones distribuidas aleatoriamente alrededor de dos centros.

In [ ]:
from sklearn.datasets import make_blobs
centers = [[2, 2], [-2, -2]]
X, labels_true = make_blobs(
    n_samples=750, centers=centers, cluster_std=0.4, random_state=0)
x = X[:,0]
y = X[:,1]

In [ ]:
(
    so.Plot(x = x, y = y)
    .add(so.Dot(), color = labels_true)
    .add(so.Line(), so.PolyFit(1))
)

En este ejemplo, podríamos decir que hay correlación entre la variable $x$ y la variable $y$?

Calculemos el R^2...

In [ ]:
modelo = linear_model.LinearRegression()    # Inicializamos un modelo de Regresion Lineal
modelo.fit(pd.DataFrame(x), y)   # Realiza

print("Coeficientes:", modelo.coef_)

# Medidas de bondad

y_pred = modelo.predict(pd.DataFrame(X[:,0]))

# Calculando el R^2
r2 = r2_score(X[:,1], y_pred)
print('R^2: ', r2)

Los datos parecen altamente correlacionados. Pero si separamos por grupo...

In [ ]:
(
    so.Plot(x = x, y = y)
    .add(so.Dot(), color = labels_true)
    .add(so.Line(), so.PolyFit(1), group = labels_true)
)

Vemos en el gráfico que la pendiente ahora pasa a ser casi 0.

¿Cómo podemos construir nosotros estos modelos y calcular los coeficientes y el R^2?

Recordemos las operaciones que nos permite hacer Formulaic.

| Operador | Ejemplo          | Función                                                                                           |
|:---------|:-----------------|:---------------------------------------------------------------------------------------------------|
| ~        | y ~ x            | Separa la variable (y) respuesta a la izquierda, de el/los predictor/es a la derecha (x).       |
| +        | y ~ x + z        | Adiciona (suma) términos al modelo.                                                              |
| :        | y ~ x : z        | Interacción entre términos. y es lineal en x ⋅ z.                                                |
| *        | y ~ x * z        | Combina adición e interacción entre términos. y ~ x * z es equivalente a y ~ x + z + x : z       |

In [ ]:
y, X = Formula("bill_length_mm ~ ???").get_model_matrix(penguins)
display(X)

In [ ]:
modelo = linear_model.LinearRegression(fit_intercept = False)    # Inicializamos un modelo de Regresion Lineal sin intercept
modelo.fit(X, y)   # Realiza
print("Coeficientes:", modelo.coef_)

**Pregunta:** ¿Cuáles coeficientes nos indican las pendientes de las rectas?

In [ ]:
# Calculamos el R^2
y_pred = modelo.predict(X)

# Calculando el R^2
r2 = r2_score(y, y_pred)
print('R^2: ', r2)

In [ ]:
# Los últimos tres números son las pendientes de las rectas.
# Cómo podemos graficar este modelo?

y_pred = modelo.predict(X)

(
    so.Plot(data = penguins, x = "bill_depth_mm", y = "bill_length_mm", color = "species")
    .add(so.Dot())
    .add(so.Line(), ???)  # Otro milagro!! No le tenía ninguna fe a esto :)
)

**Ejercicio.** Realizar ahora un modelo donde todas las rectas tengan la misma pendiente y solo cambie el intercept según la especie.